# Capstone Project: Slogan Classifier and Generator

In this capstone project you will train a Long Short-Term Memory (LSTM) model to generate slogans for businesses based on their industry, and also train a classifier to predict the industry based on a given slogan.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import spacy # available on Google Colab
from sklearn.model_selection import train_test_split

## Loading and viewing the dataset


In [6]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
data = pd.read_csv('/content/drive/MyDrive/slogan-valid.csv')
data.head()

,desc,output,type,company,industry,url,alias,desc_masked,output_masked,ent_dict,unsupported,first_pos
0,The latest <company> & Point of Sale tech for ...,Taking Care of Small Business Technology,headline_long,eftpos warehouse,computer hardware,eftposwarehouse.co.nz,Eftpos Warehouse,The latest <company> & Point of Sale tech for ...,Taking Care of Small Business Technology,{'[date]': 'monthly'},False,VB
1,Easily deliver personalized activities that en...,Build World-Class Recreation Programs,headline,welbi,"health, wellness and fitness",welbi.co,Welbi,Easily deliver personalized activities that en...,Build World-Class Recreation Programs,{},False,VB
2,Powerful lead generation software that convert...,Most Powerful Lead Generation Software for Mar...,headline_long,optinmonster,internet,optinmonster.com,Optinmonster,Powerful lead generation software that convert...,Most Powerful Lead Generation Software for Mar...,{},False,JJ
3,Twine matches companies to the best digital an...,Hire quality freelancers for your job,headline_long,twine.fm,internet,twine.fm,NaN,Twine matches companies to the best digital an...,Hire quality freelancers for your job,"{'[number]': 'over 260,000'}",False,VB
4,"Financial Advisers Norwich, Norfolk - <company...","Financial Advisers Norwich, Norfolk",headline,mcb financial services ltd,financial services,mcbfinancialservices.co.uk,Mcb Financial Services,"Financial Advisers [country], [country1] - <co...","Financial Advisers [country], [country1]","{'[country]': 'Norwich', '[country1]': 'Norfolk'}",False,NN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5346 entries, 0 to 5345
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   desc           5346 non-null   object
 1   output         5346 non-null   object
 2   type           5346 non-null   object
 3   company        5346 non-null   object
 4   industry       5346 non-null   object
 5   url            5346 non-null   object
 6   alias          4017 non-null   object
 7   desc_masked    5346 non-null   object
 8   output_masked  5346 non-null   object
 9   ent_dict       5346 non-null   object
 10  unsupported    5346 non-null   bool  
 11  first_pos      5346 non-null   object
dtypes: bool(1), object(11)
memory usage: 464.8+ KB


In [9]:
data.isna().sum()

,0
desc,0
output,0
type,0
company,0
industry,0
url,0
alias,1329
desc_masked,0
output_masked,0
ent_dict,0


In [12]:
# Extracting relevant columns and handling missing values
df = data[['industry', 'company', 'output']].dropna()
df.head()

,industry,company,output
0,computer hardware,eftpos warehouse,Taking Care of Small Business Technology
1,"health, wellness and fitness",welbi,Build World-Class Recreation Programs
2,internet,optinmonster,Most Powerful Lead Generation Software for Mar...
3,internet,twine.fm,Hire quality freelancers for your job
4,financial services,mcb financial services ltd,"Financial Advisers Norwich, Norfolk"


## Data Preprocessing

Since we are working with textual data, we need software that understands natural language. For this, we'll use a library for processing text called **spaCy**. Using spaCy, we'll break the text into smaller units called tokens that are easier for the machine to process. This process is called **tokenisation**. We'll also convert all text to lowercase and remove punctuation because this information is not necessary for our models. Run the code below, and your dataframe (df) will gain a new column called **'processed_slogan'** which contains the preprocessed text.




In [14]:
# Load spaCy model for text processing
nlp = spacy.load("en_core_web_sm")

# Define text preprocessing function
def preprocess_text(text):
    text_lower = text.lower()
    doc = nlp(text_lower)

    processed_tokens = []

    for token in doc:
        if not token.is_punct:
            processed_tokens.append(token.text)

    return " ".join(processed_tokens)

df["processed_slogan"] = df["output"].apply(preprocess_text)

df.head()

,industry,company,output,processed_slogan
0,computer hardware,eftpos warehouse,Taking Care of Small Business Technology,taking care of small business technology
1,"health, wellness and fitness",welbi,Build World-Class Recreation Programs,build world class recreation programs
2,internet,optinmonster,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...
3,internet,twine.fm,Hire quality freelancers for your job,hire quality freelancers for your job
4,financial services,mcb financial services ltd,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk


We want our model to generate **industry-specific** slogans. If we use the 'processed_slogan' column as it is, we'll be leaving out crucial context - the industries of the companies behind those slogans. To fix this, we'll create a new **'modified_slogan'** column that adds the industry name to the front of processed slogan.  

For example:  

> industry = 'computer hardware'  
processed_slogan = 'taking care of small business technology'  
modified_slogan = 'computer hardware taking care of small business technology'

Write code in the cell below to achieve this.

In [15]:

df["modified_slogan"] = df["industry"].str.lower() + " " + df["processed_slogan"]

df.head()

,industry,company,output,processed_slogan,modified_slogan
0,computer hardware,eftpos warehouse,Taking Care of Small Business Technology,taking care of small business technology,computer hardware taking care of small busines...
1,"health, wellness and fitness",welbi,Build World-Class Recreation Programs,build world class recreation programs,"health, wellness and fitness build world class..."
2,internet,optinmonster,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...,internet most powerful lead generation softwar...
3,internet,twine.fm,Hire quality freelancers for your job,hire quality freelancers for your job,internet hire quality freelancers for your job
4,financial services,mcb financial services ltd,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk,financial services financial advisers norwich ...


In [16]:
# We do not need column 'company', so we are dropping it
df = df.drop(columns=["company"])
df.head()

,industry,output,processed_slogan,modified_slogan
0,computer hardware,Taking Care of Small Business Technology,taking care of small business technology,computer hardware taking care of small busines...
1,"health, wellness and fitness",Build World-Class Recreation Programs,build world class recreation programs,"health, wellness and fitness build world class..."
2,internet,Most Powerful Lead Generation Software for Mar...,most powerful lead generation software for mar...,internet most powerful lead generation softwar...
3,internet,Hire quality freelancers for your job,hire quality freelancers for your job,internet hire quality freelancers for your job
4,financial services,"Financial Advisers Norwich, Norfolk",financial advisers norwich norfolk,financial services financial advisers norwich ...


Now we need to get data to train our model. We have textual data which we will need to represent numerically for our model to learn from it.  
The code below does the following:
1. Tokenizes a dataset of slogans.
2. Converts words to numerical indices.
3. Creates input sequences using the numerical indices.  

Here's how it works. From the 'modified_slogan' column, we take the slogan "computer hardware taking care of small business technology". The tokenisation process will convert words into their corresponding indices:  

<center>

| Word         | Token Index |
|-------------|-------|
| "computer"  | 1     |
| "hardware"  | 2     |
| "taking"    | 3     |
| "care"      | 4     |
| "of"        | 5     |
| "small"     | 6     |
| "business"  | 7     |
| "technology"| 8     |

</center>

So the tokenized list is:

<center>
[1, 2, 3, 4, 5, 6, 7, 8]
</center>

When creating input sequences for training, the loop generates progressively longer sequences.

<center>

| Token Index Sequence               | Corresponding Slogan                                 |
|------------------------------|-----------------------------------------------------|
| [1, 2]                       | "computer hardware"                                |
| [1, 2, 3]                    | "computer hardware taking"                        |
| [1, 2, 3, 4]                 | "computer hardware taking care"                   |
| [1, 2, 3, 4, 5]              | "computer hardware taking care of"                |
| [1, 2, 3, 4, 5, 6]           | "computer hardware taking care of small"          |
| [1, 2, 3, 4, 5, 6, 7]        | "computer hardware taking care of small business" |
| [1, 2, 3, 4, 5, 6, 7, 8]     | "computer hardware taking care of small business technology" |

</center>

Instead of training the model on only **complete slogans**, we provide partial phrases which will help the model learn how words connect over time. This will make it better at predicting the next word when generating slogans.  

Run the cell block below to generate the input sequences. Be sure to read the comments to understand what the code is doing.


In [17]:

# Tokenizer to convert words into numerical values tokens
tokenizer = Tokenizer()

# Tokenizer learns words in dataset
tokenizer.fit_on_texts(df["modified_slogan"])

# Total number of unique words in learned vocabulary
total_words = len(tokenizer.word_index) + 1

# Dictionary mapping words to its numerical index:
# index based on frequency i.e., more freq => lower index
tokenizer.word_index

# Creating input sequences
input_sequences = []

# Iterate over processed slogans
for line in df["modified_slogan"]:

    # Convert slogans to token sequences
    token_list = tokenizer.texts_to_sequences([line])[0]
    # returns list containing list of words indices; extracting inner list [0]
    # token_list is a list of tokenized word INDICES

    # Building list of progressively longer input sequences for better training
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

The input sequences created above are of **varying lengths**, which will be a problem when training our LSTM model. LSTMs require input sequences of **equal length**. So, we need to **pad** shorter sequences by **prepending zeros** until they match the length of the longest sequence.  

For example, if the longest sequence has **10 tokens**, our padded sequences will look like this:

<center>

| Input Sequence                     | Padded Sequence                         |
|-------------------------------------|-----------------------------------------|
| [1, 2]                              | [0, 0, 0, 0, 0, 0, 0, 0, 1, 2]         |
| [1, 2, 3]                           | [0, 0, 0, 0, 0, 0, 0, 1, 2, 3]         |
| [1, 2, 3, 4]                        | [0, 0, 0, 0, 0, 0, 1, 2, 3, 4]         |
| [1, 2, 3, 4, 5]                     | [0, 0, 0, 0, 0, 1, 2, 3, 4, 5]         |
| [1, 2, 3, 4, 5, 6]                  | [0, 0, 0, 0, 1, 2, 3, 4, 5, 6]         |
| [1, 2, 3, 4, 5, 6, 7]               | [0, 0, 0, 1, 2, 3, 4, 5, 6, 7]         |
| [1, 2, 3, 4, 5, 6, 7, 8]            | [0, 0, 1, 2, 3, 4, 5, 6, 7, 8]         |

</center>

In the cell below, write code that **finds the length of the longest sequence** in **input_sequences** and stores this value in a variable named **max_seq_len**.


In [18]:
max_seq_len = max(len(seq) for seq in input_sequences)
max_seq_len

15

Run the cell below to pad the input sequences so they are all the same length as **max_seq_length**.

In [19]:
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding="pre")

## Training Data for Slogan Generator

The input sequences generated will be used as our training data. Our LSTM needs to learn how to predict the **next word** in a sequence.  

The inputs for our model will be the input sequences **excluding the last token index** and the outputs will be the **last token index**.  

As an example, let us use the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7, 8] and say it corresponds to the slogan "computer hardware taking care of small business technology". When training the model:

> Our input **x** will be the input sequence [0, 0, 1, 2, 3, 4, 5, 6, 7] corresponding to "computer hardware taking care of small".  
> Our output **y** will be [8] which corresponds to "business".  

In the code cell below, use `input_sequences` to create the following two variables:
1. **X_gen** which contains the input sequences excluding the last token index.
2. **y_gen** which contains the last token index of the input sequence.

In [20]:
X_gen = input_sequences[:,:-1] # all columns except the last one
y_gen = input_sequences[:,-1] # only the last column

The model will output the next word of a sequence over a probability distribution. We need to encode our output variable for this to be possible.

In the code cell below, write code that will apply one-hot encoding to **y_gen** using `tf.keras.utils.to_categorical()`. **Maintain the same variable name**.  

*Hint: set the `num_classes` (number of classes) parameter to the total number of unique words in the learned vocabulary. You can access this value through a variable that was created when generating input sequences earlier.*

In [21]:
y_gen = tf.keras.utils.to_categorical(y_gen, num_classes=total_words)

## Slogan Generator Architecture

In the code cell that follows, configure the LSTM following these steps:

1. Create a sequential model using `tf.keras.models.Sequential()`. This model will have an embedding layer, two LSTM layers, and a dense output layer.
2. Add an embedding layer that converts words into dense vector representations. This layer should:
> *   Have `total_words`as the vocabulary size.
> *   Use 100 as an embedding dimension.
> *   Takes an input length of `max_seq_len - 1` (excludes the target word).
3. Add two LSTM layers.
> *   The first LSTM layer should have 150 **and** set `return_sequences` to `True`.
> *   The second LSTM layer should have 100 units.
4. Add a dense output layer which:
> *   Uses `total_words` as the number of units (one for each word in the vocabulary).
> *   Uses a softmax activation function.
5. Use `Sequential` to put everything together in the correct order to complete the architecture of the LSTM model called **gen_model**.


In [22]:
gen_model = Sequential()

# Embedding layer
gen_model.add(
    Embedding(
        input_dim=total_words,      # vocabulary size
        output_dim=100,             # embedding dimension
        input_length=max_seq_len - 1  # input sequence length (excluding target word)
    )
)

# First LSTM layer
gen_model.add(
    LSTM(
        150,
        return_sequences=True
    )
)

# Second LSTM layer
gen_model.add(
    LSTM(100)
)

# Output layer
gen_model.add(
    Dense(
        total_words,      # one unit per word in the vocab
        activation="softmax"
    )
)

gen_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In the code cell below, compile `gen_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.


In [23]:
gen_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

## Slogan Generation

In the code cell below, fit the compiled model on the inputs and outputs, setting the **number of epochs to 50**.

In [24]:
gen_model.fit(
    X_gen,
    y_gen,
    epochs=50,
    verbose=1
)

Epoch 1/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 85s 75ms/step - accuracy: 0.0618 - loss: 7.3668
Epoch 2/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 75s 69ms/step - accuracy: 0.0964 - loss: 6.4244
Epoch 3/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 75s 69ms/step - accuracy: 0.1465 - loss: 6.0492
Epoch 4/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 79s 72ms/step - accuracy: 0.1797 - loss: 5.7127
Epoch 5/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 85s 78ms/step - accuracy: 0.1986 - loss: 5.4886
Epoch 6/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 142s 78ms/step - accuracy: 0.2195 - loss: 5.2177
Epoch 7/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 83s 76ms/step - accuracy: 0.2348 - loss: 4.9813
Epoch 8/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 86s 79ms/step - accuracy: 0.2447 - loss: 4.7595
Epoch 9/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 81s 75ms/step - accuracy: 0.2566 - loss: 4.5595
Epoch 10/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 87s 79ms/step - accuracy: 0.2668 - loss: 4.3608
Epoch 11/50
1086/1086 ━━━━━━━━━━━━━━━━━━━━ 83s 76ms/step - accuracy: 0.2800 - loss: 4.1644
Epoch 1

We will now define a function called `generate_slogan` which will generate a slogan by predicting one word at a time based on a given starting phrase (the `seed_text`). This function will do this using our trained model, `gen_model`.

Here is a breakdown of how the algorithm works:  

Let us assume the dictionary mapping words to unique indices, `tokenizer.word_index`, looks like this:

> `{'computer': 1, 'hardware': 2, 'taking': 3, 'care': 4, 'of': 5}`

If the model's predicted index for the next word is 3 (`predicted_index = 3`), the loop will:

> Check 'computer' (index 1) → No match  
> Check 'hardware' (index 2) → No match  
> Check 'taking' (index 3) → Match found!  
> Assign output_word = "taking" and exit the loop.  

The `output_word` will be appended to the `seed_text`, and the process will continue to add words to the `seed_text` until we have reached the maximum number of words **or** an invalid prediction occurs.  

Carefully follow the code below and complete the missing parts as guided by the comments.

In [26]:
def generate_slogan(seed_text, max_words=20):
    for _ in range(max_words):

        # Tokenising and padding seed_text
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding="pre")

        # Predict next word probability distribution
        predictions = gen_model.predict(token_list, verbose=0)

        # Iidentify the word index with the highest probability using np.argmax()
        predicted_index = np.argmax(predictions)

        output_word = None

        # Searching for the word that corresponds to the predicted index
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # If no valid word is found, algorithm stops
        if output_word is None:
            break # out of main loop

        # Append the predicted word to seed_text and continue the process
        # until max_words is reached or an invalid prediction occurs
        seed_text = seed_text + " " + output_word

    return seed_text

## Training Data for Slogan Classifier

We will now prepare the data we will use to train our classifier. For our classifier, the inputs will come from the `processed_slogans` column of our DataFrame, `df`. The outputs will be the different industry categories under the `industry` column.

In the code cell below, extract the unique values from the `industry` column in the DataFrame and store these in a variable called **industries**.

In [27]:
industries = df["industry"].unique()
industries

array(['computer hardware', 'health, wellness and fitness', 'internet',
       'financial services', 'mechanical or industrial engineering',
       'marketing and advertising', 'hospital & health care', 'research',
       'information technology and services', 'computer software',
       'oil & energy', 'dairy', 'transportation/trucking/railroad',
       'design', 'furniture', 'professional training & coaching',
       'hospitality', 'textiles', 'food & beverages',
       'management consulting', 'medical practice', 'accounting',
       'performing arts', 'electrical/electronic manufacturing',
       'higher education', 'outsourcing/offshoring',
       'venture capital & private equity', 'writing and editing',
       'mining & metals', 'construction', 'consumer electronics',
       'retail', 'human resources', 'staffing and recruiting', 'farming',
       'wholesale', 'events services', 'import and export',
       'non-profit organization management', 'machinery',
       'information se

In [48]:
num_industries = len(industries)
num_industries

142

Create a dictionary called `industry_to_index` where each unique industry is mapped to a unique index starting from 0.

*Hint: Use the `enumerate()` function.*

In [28]:
industry_to_index = {industry: idx for idx, industry in enumerate(industries)}
industry_to_index

{'computer hardware': 0,
 'health, wellness and fitness': 1,
 'internet': 2,
 'financial services': 3,
 'mechanical or industrial engineering': 4,
 'marketing and advertising': 5,
 'hospital & health care': 6,
 'research': 7,
 'information technology and services': 8,
 'computer software': 9,
 'oil & energy': 10,
 'dairy': 11,
 'transportation/trucking/railroad': 12,
 'design': 13,
 'furniture': 14,
 'professional training & coaching': 15,
 'hospitality': 16,
 'textiles': 17,
 'food & beverages': 18,
 'management consulting': 19,
 'medical practice': 20,
 'accounting': 21,
 'performing arts': 22,
 'electrical/electronic manufacturing': 23,
 'higher education': 24,
 'outsourcing/offshoring': 25,
 'venture capital & private equity': 26,
 'writing and editing': 27,
 'mining & metals': 28,
 'construction': 29,
 'consumer electronics': 30,
 'retail': 31,
 'human resources': 32,
 'staffing and recruiting': 33,
 'farming': 34,
 'wholesale': 35,
 'events services': 36,
 'import and export': 37

Create a new column `industry_index` in your DataFrame by mapping the `industry` column to the indices using the `industry_to_index` dictionary.

*Hint: Use the  `map()` function.*

In [29]:
df["industry_index"] = df["industry"].map(industry_to_index)

Split the DataFrame `df` into training and testing sets, setting aside 20% of the data for the test set. Be sure to set the parameter `stratify=df["industry_index"]`. This ensures that both sets have the same proportion of each class (industry) as in the original dataset, resulting in balanced datasets. Call the training DataFrame `df_train` and the testing DataFrame `df_test`.

In [32]:
df_train, df_test = train_test_split(
    df,
    test_size=0.20,
    stratify=df["industry_index"],
    random_state=42
)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

Following the error above, let us check the industries that appear only once in the dataset. With stratify=..., train_test_split needs at least 2 samples per class so it can put at least 1 in train and 1 in test.

In [33]:
df["industry_index"].value_counts()

,count
industry_index,
8,452
5,267
9,257
29,195
2,187
...,...
131,1
137,1
136,1


In [41]:
df["industry_index"].value_counts()[df["industry_index"].value_counts() == 1]

,count
industry_index,
119,1
131,1
137,1
136,1
134,1
141,1


In [35]:
# We will filter the df by dropping rows of industries that appear only once in the dataset
class_counts = df["industry_index"].value_counts()
rare_classes = class_counts[class_counts < 2].index

df_filtered = df[~df["industry_index"].isin(rare_classes)]

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    df_filtered,
    test_size=0.20,
    stratify=df_filtered["industry_index"],
    random_state=42
)

Our classifier will use padded slogan sequences as inputs, similar to input sequences used for the slogan generator. The difference is we will not use sequences that get progressively longer, but instead we will use **complete slogans**. This is because our classifier does not need to learn how to predict what word comes next. It needs the full context of a slogan to learn how to accurately predict the industry.  

The next steps will walk you through how to create these sequences.  

We previously created and fitted a `Tokenizer` object called `tokenizer` while preparing data for the slogan generator. Now, we will reuse it to convert words into numerical indices.  

In the code cell below, use the `texts_to_sequences()` **method** of `tokenizer` to transform the `processed_slogan` column in **both** the `df_train` and `df_test` DataFrames into sequences of numerical indices. Store the results in variables named `X_train` and `X_test`.


In [36]:
X_train = tokenizer.texts_to_sequences(df_train["processed_slogan"])
X_test = tokenizer.texts_to_sequences(df_test["processed_slogan"])

The slogan sequences are of varying lengths. We will need to pad them the same way we did to the input sequences for the slogan generator. The `pad_sequences()` function can ensure the sequences in `slogan_sequences` have the same length.  

In the code cell below, use the `pad_sequences()` function to standardise the `slogan_sequences` lengths. Set the `maxlen` parameter to `max_seq_len`, the `padding` parameter to 0, and assign the resulting padded sequences to the same variables, `X_train` and `X_test`.

In [37]:
X_train = pad_sequences(X_train, maxlen=max_seq_len, padding="pre")
X_test = pad_sequences(X_test, maxlen=max_seq_len, padding="pre")

We have successfully created training and testing inputs for our model. Now, we will create the outputs - industry categories.

 In the code cell that follows, use `tf.keras.utils.to_categorical()` to apply one-hot encoding to the `industry_index` column of **both** `df_train` and `df_test` DataFrames. Assign the results to a variables named `y_train` and `y_test`.

 *Hint: set the `num_classes` parameter to the total number of industries in the DataFrame. The `industries` variable can be used to find this value.*

In [38]:
num_classes = len(industries)

y_train = tf.keras.utils.to_categorical(df_train["industry_index"], num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(df_test["industry_index"], num_classes=num_classes)

## Slogan Classifier Architecture

Configure the LSTM classifier following these steps:  


1. Create a Sequential model:  
   Use `tf.keras.models.Sequential()` to create a sequential model. This model will consist of an embedding layer, two LSTM layers, and a dense output layer.

2. Add an embedding layer which will convert words into dense vector representations. Configure this layer with:
   > * `total_words` as the vocabulary size.
   > * 100 as the embedding dimension.
   > * `max_seq_len` as the `input_length` (this is the length of the slogans).

3. Add the first LSTM layer. Configure it with:
   > * 150 units.
   > * Set `return_sequences` to `True` to ensure the layer outputs sequences for the next LSTM layer.

4. Add the second LSTM layer which will process the output from the previous LSTM layer. Configure it with:
   > * 100 units.
   > * No need to set `return_sequences` here (it is the final LSTM layer).

5. Add the dense output layer which will classify the data into industries. Configure it with:
   > * The number of unique industries as the number of units.
   > * The `softmax` activation function to get probabilities for each class (industry).

6. Use `Sequential` to arrange all layers in the correct order and complete the architecture of the LSTM model called **class_model**.


In [42]:
class_model = Sequential()

# Embedding layer
class_model.add(
    Embedding(
        input_dim=total_words,   # same vocab as generator
        output_dim=100,          # embedding dimension
        input_length=max_seq_len # full slogan length
    )
)

# First LSTM layer
class_model.add(
    LSTM(
        150,
        return_sequences=True
    )
)

# Second LSTM layer
class_model.add(
    LSTM(100)
)

# Output layer
class_model.add(
    Dense(
        num_classes,       # number of industries
        activation="softmax"
    )
)

class_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In the code cell below, compile `class_model` using `categorical_crossentropy` loss, an Adam optimiser, and an appropriate metric of your choice.

In [43]:
class_model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(),
    metrics=["accuracy"]
)

## Slogan Classification & Evaluation

In the code cell that follows, fit the compiled model on the inputs and outputs, setting **the number of epochs to 50**.

In [44]:
class_model.fit(
    X_train,
    y_train,
    epochs=50,
    verbose=1
)

Epoch 1/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 13s 64ms/step - accuracy: 0.0705 - loss: 4.5530
Epoch 2/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.0872 - loss: 4.2831
Epoch 3/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.0849 - loss: 4.2638
Epoch 4/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.1326 - loss: 3.9725
Epoch 5/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.2322 - loss: 3.4474
Epoch 6/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.3093 - loss: 2.9951
Epoch 7/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.3660 - loss: 2.6649
Epoch 8/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.4389 - loss: 2.3221
Epoch 9/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.5098 - loss: 2.0225
Epoch 10/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.5833 - loss: 1.7758
Epoch 11/50
134/134 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.6333 - loss: 1.5585
Epoch 12/50
134/134 ━━━━━━━━━━━━━━━━━━━

The classifier achieved a very high training accuracy (≈99.5%) and low training loss, showing that it learned the training slogans extremely well.

Evaluate the model using the testing set. Add a comment on the model's performance.

In [45]:
loss, accuracy = class_model.evaluate(X_test, y_test, verbose=1)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.1982 - loss: 7.7722
Test Loss: 7.691478252410889
Test Accuracy: 0.1994381994009018


### Comments on the classifier model perfromnace

As stated above, the classifier achieved a very high training accuracy (≈99.5%) and low training loss, showing that it learned the training slogans extremely well. However, its performance on the test set was much lower, with an accuracy of around 20% and a high loss value.

This large gap between training and test performance indicates strong overfitting. The model is memorising the training slogans rather than learning general patterns that apply to unseen slogans. Several factors contribute to this behaviour:

- The size of the dataset (142 unique industries) is relatively small compared to the model achitecture (e.g. LSTM size, number of epochs), making it easy for the model to memorise the training data.
- Many industries share similar language styles, leading to overlapping patterns that are difficult to separate.
- Some industries appear only a few times, making it hard for the model to learn reliable representations for them.
- The vocabulary is quite large, which further increases model complexity.

Overall, while the model fits the training data extremely well, it generalises poorly, and real-world predictions would be unreliable without reducing overfitting or increasing data diversity.

We will now define a function called `classify_slogan` which takes a slogan as input and predicts the industry it belongs to using the trained model, `class_model`.  

Carefully follow the code below and complete the missing parts (indicated by ellipses) as guided by the comments.

In [46]:
def classify_slogan(slogan):
    # Clean the input slogan using preprocess-text
    slogan = preprocess_text(slogan)

    # Converting the slogan to a sequence of indices
    sequence = tokenizer.texts_to_sequences([slogan])

    # Pad the sequence using the pad_sequences() function
    padded_sequence = pad_sequences(sequence, maxlen=max_seq_len, padding="pre")

    # Model prediction (probabilities)
    prediction = class_model.predict(padded_sequence, verbose=0)

    # Highest-probability class index
    predicted_index = np.argmax(prediction)

    # Return the predicted industry
    return industries[predicted_index]

## Combining the two models

Run the code cell below to combine the two models: we will first generate a slogan for a company in the "internet" industry, then pass the generated slogan to the slogan classifier to see if it correctly classifies it as internet.

In [47]:
industry = "internet"
generated_slogan = generate_slogan(industry)
predicted_industry = classify_slogan(generated_slogan)

print(f"Generated Slogan: {generated_slogan}")
print(f"Predicted Industry: {predicted_industry}")

Generated Slogan: internet web design and digital marketing glasgow portland oregon and get day sell body prep weddings reviews better self life life
Predicted Industry: leisure, travel & tourism


Compare the results and comment on any differences you notice between the generated slogans and the classifier’s predictions in the markdown cell below.


Overall, the two models do not align well in this case. The generator produces a slogan that moves away from the original and intended domain "internet" by using "body", "wedding", "life style" etc, and the classifier responds by predicting an industry that better matches the non-internet themes in the output - "leisure, travel and tourism".

Since the generated text contains little that is uniquely identifiable as internet-related beyond the first word, the classifier assigns it to the closest matching category based on its learned patterns.

This highlights the limitations of the generator’s industry conditioning and the classifier’s reliance on semantic cues present in the text.
